### mp3 -> wav -> embedding

In [1]:
import os
import numpy as np
import torch
from tortoise.api import TextToSpeech
from pydub import AudioSegment
from scipy.io.wavfile import read as read_wav

# Tortoise TTS 초기화
tts = TextToSpeech(use_deepspeed=True, kv_cache=True, half=True)

# MP3 파일이 들어있는 폴더 경로와 변환된 WAV 파일을 저장할 폴더 경로
mp3_folder = "emilyclimbs_2204_librivox"  # MP3 파일 폴더 경로
wav_folder = "emilyclimbs_wav_files"  # WAV 파일이 저장될 폴더 경로
os.makedirs(wav_folder, exist_ok=True)

# 변환된 WAV 파일을 저장할 리스트
wav_files = []

# MP3 파일을 WAV로 변환
for mp3_file in os.listdir(mp3_folder):
    if mp3_file.endswith(".mp3"):
        mp3_path = os.path.join(mp3_folder, mp3_file)
        
        # MP3를 로드하여 WAV로 변환
        audio = AudioSegment.from_mp3(mp3_path)
        audio = audio.set_frame_rate(22050).set_channels(1)  # 샘플링 레이트와 채널 설정
        wav_path = os.path.join(wav_folder, mp3_file.replace(".mp3", ".wav"))
        audio.export(wav_path, format="wav")
        wav_files.append(wav_path)
        print(f"'{mp3_file}'이 '{wav_path}'로 변환되었습니다.")

# 변환된 WAV 파일들을 로드하여 임베딩 생성
voice_samples = []
for wav_file in wav_files:
    try:
        # WAV 파일을 읽고 정규화하여 텐서로 변환
        sr, wav_audio = read_wav(wav_file)
        
        # 스테레오인 경우 모노로 변환
        if wav_audio.ndim > 1:
            wav_audio = wav_audio.mean(axis=1)
        
        # 정규화: -1 ~ 1 범위로 스케일 조정
        wav_audio = wav_audio / np.max(np.abs(wav_audio))  # 오디오 데이터를 -1에서 1 사이로 스케일 조정
        
        # torch.Tensor로 변환하고 차원 추가
        audio_tensor = torch.tensor(wav_audio, dtype=torch.float32).unsqueeze(0)  # (1, 샘플 길이)
        voice_samples.append(audio_tensor)
        print(f"WAV 파일 '{wav_file}'이 성공적으로 로드되어 텐서로 변환되었습니다.")
    except Exception as e:
        print(f"{wav_file} 로드 중 오류 발생: {e}")

# 임베딩 생성
conditioning_latents = tts.get_conditioning_latents(voice_samples)

# 임베딩을 저장할 경로
embedding_path = "saved_embeddings/conditioning_latents_emilyclimbs.npz"
os.makedirs(os.path.dirname(embedding_path), exist_ok=True)
np.savez(embedding_path, *conditioning_latents)
print(f"화자 임베딩이 '{embedding_path}'에 저장되었습니다.")


c:\Users\user\Desktop\빅데이터9기_수업자료_이수지\2024.11.04_(중간프로젝트)\tortoise-tts\tortoise\api.py:221: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.autoregressive.load_state_dict

'emilyclimbs_01_montgomery_64kb.mp3'이 'emilyclimbs_wav_files\emilyclimbs_01_montgomery_64kb.wav'로 변환되었습니다.
'emilyclimbs_02_montgomery_64kb.mp3'이 'emilyclimbs_wav_files\emilyclimbs_02_montgomery_64kb.wav'로 변환되었습니다.
'emilyclimbs_03_montgomery_64kb.mp3'이 'emilyclimbs_wav_files\emilyclimbs_03_montgomery_64kb.wav'로 변환되었습니다.
'emilyclimbs_04_montgomery_64kb.mp3'이 'emilyclimbs_wav_files\emilyclimbs_04_montgomery_64kb.wav'로 변환되었습니다.
'emilyclimbs_05_montgomery_64kb.mp3'이 'emilyclimbs_wav_files\emilyclimbs_05_montgomery_64kb.wav'로 변환되었습니다.
'emilyclimbs_06_Montgomery_64kb.mp3'이 'emilyclimbs_wav_files\emilyclimbs_06_Montgomery_64kb.wav'로 변환되었습니다.
'emilyclimbs_07_montgomery_64kb.mp3'이 'emilyclimbs_wav_files\emilyclimbs_07_montgomery_64kb.wav'로 변환되었습니다.
'emilyclimbs_08_montgomery_64kb.mp3'이 'emilyclimbs_wav_files\emilyclimbs_08_montgomery_64kb.wav'로 변환되었습니다.
'emilyclimbs_09_montgomery_64kb.mp3'이 'emilyclimbs_wav_files\emilyclimbs_09_montgomery_64kb.wav'로 변환되었습니다.
'emilyclimbs_10_montgomery_64kb.mp3'이

c:\Users\user\Desktop\빅데이터9기_수업자료_이수지\2024.11.04_(중간프로젝트)\tortoise-tts\tortoise\models\arch_util.py:314: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.mel_norms = torch

화자 임베딩이 'saved_embeddings/conditioning_latents_emilyclimbs.npz'에 저장되었습니다.


### m4b -> wav 변환

In [6]:
!ffmpeg -i GalacticPatrol2_librivox.m4b -ac 1 -ar 22050 GalacticPatrol2.wav

ffmpeg version 7.0.2-full_build-www.gyan.dev Copyright (c) 2000-2024 the FFmpeg developers
  built with gcc 13.2.0 (Rev5, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-libsnappy --enable-zlib --enable-librist --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-libbluray --enable-libcaca --enable-sdl2 --enable-libaribb24 --enable-libaribcaption --enable-libdav1d --enable-libdavs2 --enable-libuavs3d --enable-libxevd --enable-libzvbi --enable-librav1e --enable-libsvtav1 --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxavs2 --enable-libxeve --enable-libxvid --enable-libaom --enable-libjxl --enable-libopenjpeg --enable-libvpx --enable-mediafoundation --enable-libass --enable-frei0r --enable-libfreetype --enable-libfribidi --enable-libharfbuzz --enable-li

### 임베딩 저장

In [1]:
import os
import numpy as np
import torch
from tortoise.api import TextToSpeech
from scipy.io.wavfile import read as read_wav

In [2]:
# Tortoise TTS 모델 초기화
tts = TextToSpeech(use_deepspeed=True, kv_cache=True, half=True)

wav_path = "GalacticPatrol2.wav"  # 변환된 WAV 파일 경로

# WAV 파일을 로드하고 텐서로 변환
try:
    sr, wav_audio = read_wav(wav_path)
    
    # 스테레오인 경우 모노로 변환
    if wav_audio.ndim > 1:
        wav_audio = wav_audio.mean(axis=1)

    # 정규화: -1 ~ 1 범위로 스케일 조정
    wav_audio = wav_audio / np.max(np.abs(wav_audio))  # 오디오 데이터를 -1에서 1 사이로 스케일 조정

    # torch.Tensor로 변환하고 차원 추가
    audio_tensor = torch.tensor(wav_audio, dtype=torch.float32).unsqueeze(0)
    print(f"WAV 파일이 성공적으로 로드되어 텐서로 변환되었습니다.")
except Exception as e:
    print(f"WAV 파일을 로드하는 중 오류 발생: {e}")
    raise



c:\Users\user\Desktop\빅데이터9기_수업자료_이수지\2024.11.04_(중간프로젝트)\tortoise-tts\tortoise\api.py:221: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.autoregressive.load_state_dict

WAV 파일이 성공적으로 로드되어 텐서로 변환되었습니다.


In [3]:
# Tortoise TTS에서 화자 프로필 임베딩 생성
conditioning_latents = tts.get_conditioning_latents([audio_tensor])

# 임베딩 저장
embedding_path = "saved_embeddings/conditioning_latents_GalacticPatrol2"
os.makedirs(os.path.dirname(embedding_path), exist_ok=True)
np.savez(embedding_path, *conditioning_latents)
print(f"화자 임베딩이 '{embedding_path}'에 저장되었습니다.")

c:\Users\user\Desktop\빅데이터9기_수업자료_이수지\2024.11.04_(중간프로젝트)\tortoise-tts\tortoise\models\arch_util.py:314: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.mel_norms = torch

화자 임베딩이 'saved_embeddings/conditioning_latents_GalacticPatrol2'에 저장되었습니다.
